# 🌞 Example Weather App
> Using `prodb`


* Starting with a `.csv` of locations, we wish to call the [MetaWeather](https://www.metaweather.com/api/) API to pull in the weather forecast.  
* The app needs the ability to input new locations by the user

 🌍 Core functions will be converted into the streamlit app `weather_app.py`

In [ ]:
# default_exp examples

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

## 1. Requests to `metaweather` open weather API

In [ ]:
#export
#hide
import requests
import json
import arrow
import pandas as pd
from prodb.core import generate_db, insert_row

In [ ]:
#export
def get_json_from_query(location):
    """Search for a city and return metadata from API"""
    url = f"https://www.metaweather.com/api/location/search/?query={location}"
    r = requests.get(url).json()
    return r[0]


In [ ]:
l = get_json_from_query('London')
l

{'title': 'London',
 'location_type': 'City',
 'woeid': 44418,
 'latt_long': '51.506321,-0.12714'}

Check `woeid`(Where On Earth ID) is correct for London

In [ ]:
assert l['woeid'] == 44418

Get weather data for the London `woeid`

In [ ]:
utc = arrow.utcnow().format('YYYY/MM/DD')
utc

'2021/11/05'

In [ ]:
#export
def get_current_weather(location):
    """
    inputs: location str "London"
            to find woeid i.e. 44418
    """
    res = get_json_from_query(location)
    woeid = res['woeid']
    url = f"https://www.metaweather.com/api/location/{woeid}/"
    res = requests.get(url).json()
    return res

Today's forecast 🌤️

In [ ]:
%%time
res = get_current_weather('London')
res['consolidated_weather'][0]

Wall time: 1.41 s


{'id': 4750463216910336,
 'weather_state_name': 'Heavy Cloud',
 'weather_state_abbr': 'hc',
 'wind_direction_compass': 'WNW',
 'created': '2021-11-05T21:59:03.365952Z',
 'applicable_date': '2021-11-05',
 'min_temp': 2.915,
 'max_temp': 9.870000000000001,
 'the_temp': 8.695,
 'wind_speed': 4.384422413894854,
 'wind_direction': 281.959650545896,
 'air_pressure': 1027.0,
 'humidity': 65,
 'visibility': 7.677973136880617,
 'predictability': 71}

There are a number of metrological properties available to us from the API. 

The ones we are most interested in are:
* `the_temp` 🌡️ current temperature
* `weather_state_name` ⛅ current sky condition
* `max_temp` 🥵 daily max temp
* `min_temp` 🥶 daily min temp


In [ ]:
res = get_current_weather('London')
df =  pd.DataFrame.from_records(res['consolidated_weather'][:1])
df['readable_time'] = df.created.apply(lambda x: arrow.get(x).humanize())
df

,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability,readable_time
0,4750463216910336,Heavy Cloud,hc,WNW,2021-11-05T21:59:03.365952Z,2021-11-05,2.915,9.87,8.695,4.384422,281.959651,1027.0,65,7.677973,71,49 minutes ago


In [ ]:
#export
def df_from_loc(location, 
                days_ahead=1,
                keep_cols='location the_temp readable_time created applicable_date local_time latt_long weather_state_name min_temp max_temp'.split(' ')):
    res = get_current_weather(location)
    df =  pd.DataFrame.from_records(res['consolidated_weather'][:days_ahead])
    df['location'] = location
    df['local_time'] = pd.to_datetime(res['time']).strftime('%H:%M (%Y-%m-%d)')
    df['latt_long'] = res['latt_long']
    df['readable_time'] = df.created.apply(lambda x: arrow.get(x).humanize())
    return df[keep_cols]


In [ ]:
df = df_from_loc('London', days_ahead=2)
df

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,London,8.695,49 minutes ago,2021-11-05T21:59:03.365952Z,2021-11-05,22:48 (2021-11-05),"51.506321,-0.12714",Heavy Cloud,2.915,9.87
1,London,11.945,49 minutes ago,2021-11-05T21:59:02.455721Z,2021-11-06,22:48 (2021-11-05),"51.506321,-0.12714",Showers,7.010,13.07


Example output showing location and current weather situation

In [ ]:
df = df_from_loc('Auckland')
df

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,Auckland,19.185,11 minutes ago,2021-11-05T22:37:05.593308Z,2021-11-06,11:48 (2021-11-06),"-36.884109,174.770416",Heavy Rain,12.97,20.13


# 🦄 Pro db 

In [ ]:
import sys
sys.path.append('../')

from prodb.core import generate_db, insert_row

### Initialise empty db

In [ ]:
dbpath = 'weather_db.csv'
cols = 'location temp high low weather_state'.split()

generate_db(cols=cols, dbpath=dbpath)
df = pd.read_csv('db.csv')
df.head()

✓💾 weather_db.csv (38 kb)


,name,mood,message,time_utc,human_time,high,local_time,location,low,temp,weather_state
0,Sam,😊,"hello from London, UK",2021-11-05 22:29:05,40 seconds ago,NaN,NaN,NaN,NaN,NaN,NaN
1,Grant,😵,"hello from Christchurch, NZ",2021-11-05 22:29:05,40 seconds ago,NaN,NaN,NaN,NaN,NaN,NaN
2,Luke,👹,"hello from London, UK",2021-11-05 22:29:05,40 seconds ago,NaN,NaN,NaN,NaN,NaN,NaN
3,Bill,👹,hi,2021-11-05 22:29:05,40 seconds ago,NaN,NaN,NaN,NaN,NaN,NaN
4,Luke,😊,"hello, from UK",2021-11-05 22:29:06,39 seconds ago,NaN,NaN,NaN,NaN,NaN,NaN


## Add row

In [ ]:
dx = df_from_loc('Auckland')
dx

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,Auckland,19.185,11 minutes ago,2021-11-05T22:37:05.593308Z,2021-11-06,11:48 (2021-11-06),"-36.884109,174.770416",Heavy Rain,12.97,20.13


In [ ]:
dx = df_from_loc('Christchurch')

data = {'location': dx.location.item(),
        'time_utc': arrow.utcnow().format('YYYY-MM-DD HH:mm:ss'),
        'temp': dx.the_temp.item(),
        'high': dx.max_temp.item(),
        'low': dx.min_temp.item(),
        'weather_state': dx.weather_state_name.item(),
        'local_time': dx.local_time.item()}

df = insert_row(df, data)
display(df)

,name,mood,message,time_utc,human_time,high,local_time,location,low,temp,weather_state
0,Sam,😊,"hello from London, UK",2021-11-05 22:29:05,40 seconds ago,NaN,NaN,NaN,NaN,NaN,NaN
1,Grant,😵,"hello from Christchurch, NZ",2021-11-05 22:29:05,40 seconds ago,NaN,NaN,NaN,NaN,NaN,NaN
2,Luke,👹,"hello from London, UK",2021-11-05 22:29:05,40 seconds ago,NaN,NaN,NaN,NaN,NaN,NaN
3,Bill,👹,hi,2021-11-05 22:29:05,40 seconds ago,NaN,NaN,NaN,NaN,NaN,NaN
4,Luke,😊,"hello, from UK",2021-11-05 22:29:06,39 seconds ago,NaN,NaN,NaN,NaN,NaN,NaN
5,Luke,😊,"hello, from UK",2021-11-05 22:29:10,35 seconds ago,NaN,NaN,NaN,NaN,NaN,NaN
6,Luke,😊,"hello, from UK",2021-11-05 22:29:36,just now,NaN,NaN,NaN,NaN,NaN,NaN
7,Luke,😊,"hello, from UK",2021-11-05 22:29:40,just now,NaN,NaN,NaN,NaN,NaN,NaN
8,Luke,😊,"hello, from UK",2021-11-05 22:29:44,just now,NaN,NaN,NaN,NaN,NaN,NaN
9,Luke,😊,"hello, from UK",2021-11-05 22:29:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#export

def visit_city(df, cities, dbpath):
    if isinstance(cities, str): cities = [cities]
    for city in cities:
        dx = df_from_loc(city).round(1)

        data = {'location': dx.location.item(), 
                'time_utc': arrow.utcnow().format('YYYY-MM-DD HH:mm:ss'),
                'temp': dx.the_temp.item(),
                'high': dx.max_temp.item(),
                'low': dx.min_temp.item(),
                'weather_state': dx.weather_state_name.item(),
                'local_time': dx.local_time.item(),
                'latlong': dx.latt_long.item()}

        df = insert_row(df, data, dbpath)
    return df.round(1)
        

In [ ]:
df = visit_city(df, ['Wellington', 'Lagos', 'Zagreb'], dbpath)
display(df)

In [ ]:
df = visit_city(df, 'Wellington', dbpath)
display(df)

In [ ]:
df = visit_city(df, ['Singapore', 'Alexandria', 'Bangkok'], dbpath)
display(df)